In [1]:
%matplotlib qt5

# Preparation for programming
# Make sure to execute this cell first!
import os
import glob
import warnings
warnings.filterwarnings('ignore')                  # do not show warnings
from obspy import read, Stream
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pylab as plt
import os
from os import path
import pandas as pd
from datetime import datetime 
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.geodetics import kilometers2degrees
from plotw_rs import plotw_rs
from matplotlib import dates
import datetime
from matlab2datetime import matlab2datetime
from obspy.core import read, UTCDateTime
import numpy as np
from obspy.geodetics import gps2dist_azimuth

# Python adaptation of run_getwaveform_626.m written by Carl Tape for Applied Seismology,GEOS 626, 
# Python coding done by Nealey Sims
#
# Load waveforms from the UAF waveform database, then plot record sections.
#
# copied from run_getwaveform_short.m in GEOTOOLS
# Abbreviated version of run_getwaveform.m and getwaveform_input.m
#

#==========================================================================
# USER PARAMETERS

iex = 1                # CHANGE THIS OR ADD YOU OWN EXAMPLE
bgetwaveform = True    # 
bplotrs = True         # 

#==========================================================================

spdy = 86400

# default parameters for plotting record sections
# note: you can over-ride these within each example below
rssort = 2      # =1 by azimuth, =2 by distance
iabs = 0
T1 = []
T2 = []
trshift = 0
tmark = []
pmax = 50
iintp = 0
inorm = [1]
nfac = 1
azstart = []
iunit = 1
tlims = []     # time limits for plotting
imap = 1

#==========================================================================

# default parameters for waveform extraction
samplerate = []
cutoff = []

if iex == 1:
    #Yahtse event recorded within two databases (AEC, Yahtse)
    #idatabase = 5; stasub = [-141.489 -141.377 60.205 60.245];
    idatabase = [1, 5]; stasub = [0 ,200];

    # source parameters (some can be empty)
    #originTime = datenum('2010/09/18 14:15:02')
    originTime = dates.date2num(datetime.datetime(2010,9,18,14,15,2))
    elat = 60.155496
    elon = -141.378343
    edep_km = 0
    eid = []
    mag = []

    chan = ["HHZ","BHZ"]

    duration_s = 70
    oshift = 20
    T1 = [0.1]
    T2 = [2]
elif iex == 2:
    # Mw 7.5 SE Alaska
    idatabase = 1; stasub = [0, 500];
    # source parameters (some can be empty)
    #originTime = datenum('2013/01/05 08:58:32.4')
    originTime = dates.date2num(datetime.datetime(2013,1,5,8,58,32,4))
    elat = 55.62
    elon = -135.13
    edep_km = 0
    eid = []
    mag = []
    # broadband channels
    chan1 = ['BHZ','BHE','BHN','BH1','BH2']
    # strong motion channels
    chan2 = ['BNZ','BNE','BNN','BN1','BN2','BLZ','BLE','BLN','BL1','BL2',
            'HNZ','HNE','HNN','HN1','HN2','HLZ','HLE','HLN','HL1','HL2']
    # warning: waveforms will have different units (nm/s, nm/s^2)
    chan = chan1+ chan2
    
    duration_s = 300
    oshift = 50
    
elif iex == 3:
    # explosion in Fairbanks
    idatabase = 1;
    # source parameters (some can be empty)
    #originTime = datenum('2013/02/03 01:10:31.495')
    originTime = dates.date2num(datetime.datetime(2013,2,3,1,10,31,495))
    #elat = 64.8156; elon = -147.9419;  % original AEC
    #elat = 64.8045; elon = -147.9653;  % reviewed AEC
    elat = 64.80175; elon = -147.98236 # infrasound
    edep_km = 0
    eid = []
    mag = []
    # broadband channels
    chan = ['SHZ','HHZ','BHZ']
    stasub = [0, 200]
    duration_s = stasub(2)/0.30  # air wave
    oshift = 50
    # record section
    T1 = 0.2
    T2 = 1
    
elif iex == 4:
    # very low frequency (VLF) event near Kantishna
    idatabase = 1;
    # source parameters (some can be empty)
    #originTime = datenum('2014/01/22 12:14:34');
    originTime = dates.date2num(datetime.datetime(2014,1,22,12,14,34))
    elat = 63.463; elon = -150.109
    edep_km = 38
    eid = []
    mag = 1.7
    # broadband channels
    chan = ['BHZ']
    stasub = [0, 200]
    duration_s = 100 
    oshift = 0
    T1 = []; T2 = 2
    
elif iex == 5:
    # landslide near Lituya
    idatabase = 1; stasub = [0 ,1000];
    # source parameters (some can be empty)
    #originTime = datenum('2014/02/16 14:24:00')
    originTime = dates.date2num(datetime.datetime(2014,2,16,14,24,0))
    elat = 58.68; elon = -137.37
    edep_km = 0
    eid = []
    mag = []
    chan = ['BHZ']  # consider HHZ as well
    duration_s = 600
    oshift = 0
    T1 = 10; T2 = 40
    pmax = 40
    
elif iex == 6:
    # Sumatra Mw 8.6 in Alaska and triggered earthquakes
    idatabase = 1
    # four different events
    #originTimeS = datenum('2012/04/11 08:38:37.30');    # Sumatra (CMT-PDE)
    originTimeS = dates.date2num(datetime.datetime(2012,4,11,8,38,37,30))
    #originTimeA = datenum('2012/04/11 09:00:09.71');    # Andreanof (NEIC)
    originTimeA = dates.date2num(datetime.datetime(2012,4,11,9,0,9,71))
    #originTimeN = datenum('2012/04/11 09:21:57.44');    # Nenana (NEIC)
    originTimeN = dates.date2num(datetime.datetime(2012,4,11,9,21,57,44))
    #originTimeI = datenum('2012/04/11 09:40:58.02');    # Iliamna slab (NEIC)
    originTimeI = dates.date2num(datetime.datetime(2012,4,11,9,40,58,2))
    elatS = 2.24; elonS = 92.78; edepS = 40.03; emagS = 8.6
    elatA = 51.36; elonA = -176.10; edepA = 20; emagA = 5.5
    elatN = 64.9222; elonN = -148.9461; edepN = 19.4; emagN = 3.88
    elatI = 60.10; elonI = -152.83; edepI = 101; emagI = 2.9
    
    # source
    elat = elatS; elon = elonS; edep_km = edepS; mag = emagS
    eid = '201204110838A';  # GCMT
    originTime = originTimeS;
    chan = ['BHZ']
    dmax_deg = 25
    stasub = [elonN, elatN, 0, dmax_deg, 0, 360]
    
    # parameters for Sumatra waveforms across Alaska
    duration_s = 3600*2.0
    oshift = 3600*0.25;
    T1 = 1/4; T2 = 1/2;     # P wave + triggered events
    #T1 = 2; T2 = 1000;     % full wavetrain (no triggered events visible)
    iunit = 2
    # three different triggered events are visible
    tmark = [originTimeA, originTimeN, originTimeI]
    
elif iex == 7:
    # one of the triggered earthquakes
    idatabase = 1
    # four different events
    #originTimeS = datenum('2012/04/11 08:38:37.30');    # Sumatra (CMT-PDE)
    originTimeS = dates.date2num(datetime.datetime(2012,4,11,8,38,37,30))
    #originTimeA = datenum('2012/04/11 09:00:09.71');    # Andreanof (NEIC)
    originTimeA = dates.date2num(datetime.datetime(2012,4,11,9,0,9,71))
    #originTimeN = datenum('2012/04/11 09:21:57.44');    # Nenana (NEIC)
    originTimeN = dates.date2num(datetime.datetime(2012,4,11,9,21,57,44))
    #originTimeI = datenum('2012/04/11 09:40:58.02');    # Iliamna slab (NEIC)
    originTimeI = dates.date2num(datetime.datetime(2012,4,11,9,40,58,2))
    elatS = 2.24; elonS = 92.78; edepS = 40.03; emagS = 8.6
    elatA = 51.36; elonA = -176.10; edepA = 20; emagA = 5.5
    elatN = 64.9222; elonN = -148.9461; edepN = 19.4; emagN = 3.88
    elatI = 60.10; elonI = -152.83; edepI = 101; emagI = 2.9
    
    # source: USER CHOOSE ONE
    originTime = originTimeN; elat = elatN; elon = elonN; edep_km = edepN; # Nenana
    mag = emagN; stasub = [0 , 200]; duration_s = 200;  
    #originTime = originTimeA; elat = elatA; elon = elonA; edep_km = edepA; # Andreanof
    #mag = emagA; stasub = [0 , 2000]; duration_s = 600;  
    #originTime = originTimeI; elat = elatI; elon = elonI; edep_km = edepI; # Iliamna
    #mag = emagI; stasub = [0 , 400]; duration_s = 200;  
    eid = []
    chan = ['BHZ']

    # bandpass
    oshift = 10; T1 = 1/4; T2 = 1/2
#elif iex == 8: 
    # TRY YOUR OWN EXAMPLE HERE
    
# tshift (and trshift) are for plotting record sections only
tshift = [oshift + trshift]

#startTime = originTime - max(oshift)/spdy;
startTime = originTime - oshift/spdy
endTime   = originTime + duration_s/spdy
dur_dy = endTime-startTime
print('origin time is %s\n' % (dates.num2date(originTime)))
print('startTime is %s\n' % (dates.num2date(startTime)))
print('total length of time requested: %.2f s (= %.2f min = %.2f hours)\n' %
    (dur_dy*spdy,dur_dy*3600,dur_dy*24))

# additional user parameters
#sacdir = './';      # =[] to return waveform object only
sacdir = []
iint = 0            # integrate waveforms: =1 for displacement, =0 for velocity
iprocess = 1        # iprocess = 2 to deconvolve
irs = 1

origin time is 2010-09-18 14:15:02+00:00

startTime is 2010-09-18 14:14:42+00:00

total length of time requested: 90.00 s (= 3.75 min = 0.02 hours)



In [3]:
# Setting up parameters for getting waveforms
client = Client("IRIS")
wvfrm_download = False # Run True once if running example for first time.
starttime = originTime-oshift/spdy  
endtime = originTime+duration_s/spdy

t = UTCDateTime(dates.num2date(originTime))   # 
starttime = UTCDateTime(dates.num2date(starttime))
endtime = UTCDateTime(dates.num2date(endtime))
#starttime = t-(1*24*60*60)                 # 1 day before the the origin time
#endtime = t+(9*24*60*60) 
# Measure the download time
#max radius for station search
maxr=kilometers2degrees(stasub[1])
#cat = client.get_events(starttime=starttime, endtime=endtime)#,catalog="ISC")
'''
try:
    cat = client.get_events(starttime=starttime, endtime=endtime)
except Exception as e:
    print(e)

print(cat)
'''
# Download and save the raw waveform in a new directory
directory = "./data"
if not path.exists(directory):  # If data directory doesn't exist, it will create one
    os.makedirs(directory)
if wvfrm_download == True:      # clear directory to avoid using wrong stations
    files = glob.glob(directory)
    for f in files:
        os.remove(f)
inventory = Inventory()
st = Stream()
traces=[]
slat=[]
slon=[]
stanames=[]
loc=[]
stas=[]
ST=Stream()
for chan_code in chan: 
    try:
        inventory += client.get_stations(longitude=elon, latitude=elat,
                            minradius=stasub[0], maxradius=maxr, #network="XF,AK",
                            starttime=starttime, endtime =endtime,  
                            channel=chan_code, level="response")
    except:
        print("No data for channel %s " % (chan_code))
print (inventory)
for net in inventory:
    for sta in net:
        stanames.append(sta.code)
        cha=sta[0]
        slat.append(sta.latitude)
        slon.append(sta.longitude)
        loc.append(cha.location_code)
if wvfrm_download == True:
     
    for network in inventory:
        for station in network:
            for chan_code in chan:
                try:
                    st = client.get_waveforms(network.code, station.code, "*", chan_code,
                                               starttime, endtime, attach_response=True)
                    
                    st.write(path.join(directory, station.code), format = 'SAC')
                    stas.append(station.code)
                except:
                    pass


    st.write("example.mseed", format="MSEED")

st1 = read('data/*',header=None )
        
for i, tr in enumerate(st1):
    for s, stan in enumerate(stanames):
        
        if str(tr.stats.station) == str(stan): 
            tr.stats.sac.stla=slat[s]
            tr.stats.sac.stlo=slon[s]
            tr.stats.distance = gps2dist_azimuth(tr.stats.sac.stla, tr.stats.sac.stlo, elat, elon)[0]
            tr.stats.location = loc[0]
            
            traces.append(tr)
            ST.append(tr)
testt = plotw_rs(ST, elat, elon, rssort=rssort, iabs=0, tshift=tshift, tmark=[], T1=T1, T2=T2, pmax=50, inorm=[1])
   

Inventory created at 2021-01-08T10:31:59.048392Z
	Created by: ObsPy 1.2.2
		    https://www.obspy.org
	Sending institution: IRIS-DMC,ObsPy 1.2.2 (IRIS-DMC)
	Contains:
		Networks (7):
			5C, AK, AT, CN, XF, XZ, YO
		Stations (59):
			5C.HRLQ (Harlequin Lake, Alaska)
			AK.BAL (Baldy Mountain, AK, USA)
			AK.BGLC (Bering Glacier BLM Camp, AK, USA)
			AK.BMR (Bremner River, AK, USA)
			AK.CRQ (Cirque, AK, USA)
			AK.CTG (Chitna Glacier, AK, USA)
			AK.NICH (Nichawak Mountain, AK, USA)
			AK.PIN (Pinnacle, AK, USA)
			AK.PNL (Peninsula, AK, USA)
			AK.RAG (Ragged Mountain, AK, USA)
			AK.TGL (Tana Glacier, AK, USA)
			AT.YKU2 (Yakutat, Alaska)
			CN.YUK2 (White River, YT, CA)
			CN.YUK3 (Moose Creek, YT, CA)
			CN.YUK6 (Outpost Mountain, YT, CA)
			CN.YUK7 (Dusty Glacier, YT, CA)
			XF.BOOM (Booming Terrace)
			XF.CNTR (Center. (On ice. Sept. 2010 location))
			XF.COAL (Coal Ridge)
			XF.COED (COED (On ice. Sept. 2010 location))
			XF.DOST (Downstream)
			XF.FUFU (FUFU (On ice. Sept. 2010 